In [2]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Einlesen aller HDF5-Files

In [8]:
zamg = pd.read_hdf('../data/zamg/zamg.h5') 
zamg["date"] = pd.to_datetime(zamg["date"])
print(zamg.dtypes)
print(zamg)

location            object
date        datetime64[ns]
rain               float64
deg                float64
dtype: object
    location       date  rain   deg
0    Güssing 2022-05-23   0.0  19.6
1    Güssing 2022-05-24  15.1  19.7
2    Güssing 2022-05-25  16.5  17.6
3    Güssing 2022-05-26   0.0  18.1
4    Güssing 2022-05-27  12.0  20.1
..       ...        ...   ...   ...
95  Poysdorf 2022-06-11   0.0  20.4
96  Poysdorf 2022-06-12   0.0  21.2
97  Poysdorf 2022-06-13   0.1  18.4
98  Poysdorf 2022-06-14   0.0  17.2
99  Poysdorf 2022-06-15   0.0  17.9

[100 rows x 4 columns]


In [19]:
ecadRR = pd.read_hdf('../data/ecad/RR.h5') 
ecadRR["DATE"] = pd.to_datetime(ecadRR["DATE"], format='%Y%m%d')

# add station name
ecadRR.loc[ecadRR["SOUID"] == 234717 , "name"] = "KREMS"
ecadRR.loc[ecadRR["SOUID"] == 234938 , "name"] = "NEUSIEDL AM SEE"
ecadRR.loc[ecadRR["SOUID"] == 235003 , "name"] = "POYSDORF"
ecadRR.loc[ecadRR["SOUID"] == 236251 , "name"] = "GÜSSING"

print(ecadRR.dtypes)
print(ecadRR)

ecadTG = pd.read_hdf('../data/ecad/TG.h5') 
ecadTG["DATE"] = pd.to_datetime(ecadTG["DATE"], format='%Y%m%d')

# add station name
ecadTG.loc[ecadTG["SOUID"] == 234722 , "name"] = "KREMS"
ecadTG.loc[ecadTG["SOUID"] == 234943 , "name"] = "NEUSIEDL AM SEE"
ecadTG.loc[ecadTG["SOUID"] == 235008 , "name"] = "POYSDORF"
ecadTG.loc[ecadTG["SOUID"] == 236256 , "name"] = "GÜSSING"

print(ecadTG.dtypes)
print(ecadTG)

SOUID             int64
DATE     datetime64[ns]
RR                int64
name             object
dtype: object
        SOUID       DATE  RR     name
0      234717 1936-01-02   1    KREMS
1      234717 1936-01-03   3    KREMS
2      234717 1936-01-04  44    KREMS
3      234717 1936-01-05  11    KREMS
4      234717 1936-01-06   0    KREMS
...       ...        ...  ..      ...
99910  236251 2021-12-27   0  GÜSSING
99911  236251 2021-12-28  41  GÜSSING
99912  236251 2021-12-29   6  GÜSSING
99913  236251 2021-12-30   1  GÜSSING
99914  236251 2021-12-31   0  GÜSSING

[99915 rows x 4 columns]
SOUID             int64
DATE     datetime64[ns]
TG                int64
name             object
dtype: object
        SOUID       DATE  TG     name
0      234722 1936-01-01  38    KREMS
1      234722 1936-01-02  33    KREMS
2      234722 1936-01-03  34    KREMS
3      234722 1936-01-04  51    KREMS
4      234722 1936-01-05  36    KREMS
...       ...        ...  ..      ...
98455  236256 2021-12-27  12  GÜ

array(['KREMS', 'NEUSIEDL AM SEE', 'POYSDORF', 'GÜSSING'], dtype=object)

# Basis-Visualisierung für alle Datensätze
## GeDaBa Gemeinden im Umkreis der Wetterstationnen
# ZAMG-Vergleich mit ECAD Temperatur-Daten und Regen
# GeDaBa-Daten (ausgewählte Gemeinden) und ECAD-Daten beschreiben